# Pulling Data from Google BigQuery

In [1]:
import pandas as pd 
import numpy as np 
from google.cloud import bigquery

In [2]:
# Construct a BigQuery client object to pull data from BigQuery
client = bigquery.Client(project='cyclistic-pipeline-project')

# The query pulls Citibike trip data from 2018, joining with weather data and zip code information
query = """
    SELECT
        TRI.usertype,
        ZIPSTART.zip_code AS zip_code_start,
        ZIPSTARTNAME.borough borough_start,
        ZIPSTARTNAME.neighborhood AS neighborhood_start,
        ZIPEND.zip_code AS zip_code_end,
        ZIPENDNAME.borough borough_end,
        ZIPENDNAME.neighborhood AS neighborhood_end,
        -- Since we're using trips in 2018, we will add 5 years to make it look recent
        TIMESTAMP_ADD(TRI.starttime, INTERVAL 5 YEAR) AS start_time,
        TIMESTAMP_ADD(TRI.stoptime, INTERVAL 5 YEAR) AS stop_time,
        WEA.temp AS day_mean_temperature, -- Mean temp
        WEA.wdsp AS day_mean_wind_speed, -- Mean wind speed
        WEA.prcp day_total_precipitation, -- Total precipitation
        -- Group trips into 10 minute intervals to reduces the number of rows
        ROUND(CAST(TRI.tripduration / 60 AS INT64), -1) AS trip_minutes,
        AVG(TRI.tripduration) / 60 AS avg_trip_minutes
    
    FROM `bigquery-public-data.new_york_citibike.citibike_trips` AS TRI
    
    INNER JOIN `bigquery-public-data.geo_us_boundaries.zip_codes` ZIPSTART
    ON ST_WITHIN(
    ST_GEOGPOINT(TRI.start_station_longitude, TRI.start_station_latitude),
    ZIPSTART.zip_code_geom)
    
    INNER JOIN `bigquery-public-data.geo_us_boundaries.zip_codes` ZIPEND
    ON ST_WITHIN(
    ST_GEOGPOINT(TRI.end_station_longitude, TRI.end_station_latitude),
    ZIPEND.zip_code_geom)
    
    INNER JOIN `bigquery-public-data.noaa_gsod.gsod20*` AS WEA
    ON PARSE_DATE("%Y%m%d", CONCAT(WEA.year, WEA.mo, WEA.da)) = DATE(TRI.starttime)
    
    INNER JOIN `cyclistic-pipeline-project.cyclistics.zipcodes` AS ZIPSTARTNAME
    ON ZIPSTART.zip_code = CAST(ZIPSTARTNAME.zip AS STRING)
    
    INNER JOIN `cyclistic-pipeline-project.cyclistics.zipcodes` AS ZIPENDNAME
    ON ZIPEND.zip_code = CAST(ZIPENDNAME.zip AS STRING)
    
    WHERE
        -- This takes the weather data from one weather station
        WEA.wban = '94728' -- NEW YORK CENTRAL PARK
        -- Use data in 2018
        AND EXTRACT(YEAR FROM DATE(TRI.starttime)) = 2018 
    
    GROUP BY 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13

"""

query_job = client.query(query) 
df_trips = query_job.to_dataframe()

df_trips.to_csv('../data/trips.csv', index=False)

/opt/anaconda3/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [4]:
# Pull station data
query = """
    SELECT *
    FROM `bigquery-public-data.new_york_citibike.citibike_stations`
"""

query_job = client.query(query) 
df_station = query_job.to_dataframe()

df_station.to_csv('../data/stations.csv', index=False)

/opt/anaconda3/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
